<a target="_parent" href="https://colab.research.google.com/github/gretelai/gretel-blueprints/blob/main/docs/notebooks/demo/navigator/rag-examples/generate-rag-evaluation-dataset.ipynb">
  <img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/>
</a>

# 🎨 Navigator Data Designer SDK: Generate Diverse RAG Evaluations

Generate comprehensive evaluation datasets for your RAG systems, customized to your content and use cases. This blueprint helps you create diverse question-answer pairs at scale, testing both answerable and unanswerable scenarios across different difficulty levels and reasoning types that form a critical step in deploying a production grade RAG system.

> **Note:** The [Data Designer](https://docs.gretel.ai/create-synthetic-data/gretel-data-designer-beta) functionality demonstrated in this notebook is currently in **Early Preview**. To access these features and run this notebook, please [join the waitlist](https://gretel.ai/navigator/data-designer#waitlist).

# 📘 Getting Started

First, let's install and import the required packages:

In [1]:
# Install required libraries

!pip install -qq langchain smart_open git+https://github.com/gretelai/gretel-python-client
!pip install 'unstructured[pdf]'

In [2]:
# Configuration
# -------------
# Define source documents and total number of evaluation pairs to generate.
# You can replace this with your own documents.

DOCUMENT_LIST = ["https://gretel-public-website.s3.us-west-2.amazonaws.com/datasets/rag_evals/databricks-state-of-data-ai-report.pdf"]
NUM_EVALS = 100

In [3]:
# Document Processing
# ------------------
# The DocumentProcessor class handles loading and chunking source documents for RAG evaluation.
# We use langchain's RecursiveCharacterTextSplitter and unstructured.io for robust document parsing.

from typing import List, Union
from langchain.text_splitter import RecursiveCharacterTextSplitter
from unstructured.partition.auto import partition
from smart_open import open
import tempfile
import os

class DocumentProcessor:
    def __init__(self, chunk_size: int = 4192, chunk_overlap: int = 200):
        self.text_splitter = RecursiveCharacterTextSplitter(
            chunk_size=chunk_size,
            chunk_overlap=chunk_overlap,
            length_function=len,
        )

    def parse_document(self, uri: str) -> str:
        """Parse a single document from URI into raw text."""
        with open(uri, 'rb') as file:
            content = file.read()
            with tempfile.NamedTemporaryFile(delete=False) as temp_file:
                temp_file.write(content)
                temp_file.flush()
                elements = partition(temp_file.name)

        os.unlink(temp_file.name)
        return "\n\n".join([str(element) for element in elements])

    def process_documents(self, uris: Union[str, List[str]]) -> List[str]:
        """Process one or more documents into chunks for RAG evaluation."""
        if isinstance(uris, str):
            uris = [uris]

        all_chunks = []
        for uri in uris:
            text = self.parse_document(uri)
            chunks = self.text_splitter.split_text(text)
            all_chunks.extend(chunks)

        return all_chunks

In [4]:
# Data Models
# -----------
# Define Pydantic models for structured output generation:
# 1. QAPair: Schema for question-answer evaluation pairs
# 2. EvalMetrics: Schema for scoring generation quality

from pydantic import BaseModel, Field
from typing import Optional, Literal

class QAPair(BaseModel):
    question: str = Field(..., description="A specific question related to the domain of the context")
    answer: str = Field(..., description="Either a context-supported answer or explanation of why the question cannot be answered")


class EvalMetrics(BaseModel):
   context_relevance: int = Field(..., description="How relevant the retrieved context is (1=irrelevant, 5=perfectly relevant)", ge=1, le=5)
   answer_precision: int = Field(..., description="Answer accuracy or appropriateness (1=incorrect/inappropriate, 5=perfect)", ge=1, le=5)
   answer_completeness: int = Field(..., description="Information completeness (1=missing critical info, 5=fully complete)", ge=1, le=5)
   hallucination_avoidance: int = Field(..., description="Adherence to facts (1=complete fabrication, 5=no hallucination)", ge=1, le=5)

In [5]:
# Setup & Configure Data Designer using the new SDK
# --------------------------------
# Process document chunks

import pandas as pd
from gretel_client.navigator_client import Gretel

# Process document chunks
processor = DocumentProcessor(chunk_size=4192, chunk_overlap=200)
chunks = processor.process_documents(DOCUMENT_LIST)

# Create a seed DataFrame with the document chunks
seed_df = pd.DataFrame({"context": chunks})

In [ ]:
# Initialize Gretel client and Data Designer
gretel = Gretel(api_key="prompt", endpoint="https://api.dev.gretel.ai")
aidd = gretel.data_designer.new(model_suite="llama-3.x")

# Upload the seed dataset with document chunks
aidd.with_seed_dataset(seed_df, sampling_strategy="shuffle", with_replacement=True)

In [ ]:
# Add categorical columns for controlled diversity
aidd.add_column(
    name="difficulty",
    type="category",
    params={"values": ["easy", "medium", "hard"]}
)

# Add reasoning_type as a subcategory of difficulty
aidd.add_column(
    name="reasoning_type",
    type="category",
    params={"values": [
        "factual recall",
        "inferential reasoning",
        "comparative analysis",
        "procedural understanding",
        "cause and effect"
    ]}
)

# Add question_type with weights to generate more answerable questions than unanswerable
aidd.add_column(
    name="question_type",
    type="category",
    params={"values": [
        "answerable",
        "unanswerable"
    ], "weights": [10, 1]}
)

In [ ]:

# Add Q&A pair generation column
aidd.add_column(
    name="qa_pair",
    type="llm-structured",
    system_prompt="""You are an expert at generating high-quality RAG evaluation pairs.
Your output should include both answerable and unanswerable questions to properly test RAG systems.""",
    prompt="""\n{{context}}\n\n
Generate a {{difficulty}} {{reasoning_type}} question-answer pair.
The question should be {{question_type}} using the provided context.

For answerable questions:
- Ensure the answer is fully supported by the context
- Make the reasoning clear and traceable

For unanswerable questions:
- Keep the question topically relevant
- Make it clearly beyond the context's scope
- Explain why it cannot be answered

Put your thoughts within <think>...</think> before providing the JSON.""",
    output_format=QAPair
)

In [ ]:
from gretel_client.data_designer.params import Rubric

# Define evaluation rubrics
context_relevance_rubric = Rubric(
    name="Context Relevance",
    description="Evaluates how relevant the answer is to the provided context",
    scoring={
        "5": "Perfect relevance to context with no extraneous information",
        "4": "Highly relevant with minor deviations from context",
        "3": "Moderately relevant but includes some unrelated information",
        "2": "Minimally relevant with significant departure from context",
        "1": "Almost entirely irrelevant to the provided context"
    }
)

answer_precision_rubric = Rubric(
    name="Answer Precision",
    description="Evaluates the accuracy and specificity of the answer",
    scoring={
        "5": "Extremely precise with exact, specific information",
        "4": "Very precise with minor imprecisions",
        "3": "Adequately precise but could be more specific",
        "2": "Imprecise with vague or ambiguous information",
        "1": "Completely imprecise or inaccurate"
    }
)

answer_completeness_rubric = Rubric(
    name="Answer Completeness",
    description="Evaluates how thoroughly the answer addresses all aspects of the question",
    scoring={
        "5": "Fully complete, addressing all aspects of the question",
        "4": "Mostly complete with minor omissions",
        "3": "Adequately complete but missing some details",
        "2": "Substantially incomplete, missing important aspects",
        "1": "Severely incomplete, barely addresses the question"
    }
)

hallucination_avoidance_rubric = Rubric(
    name="Hallucination Avoidance",
    description="Evaluates the absence of made-up or incorrect information",
    scoring={
        "5": "No hallucinations, all information is factual and verifiable",
        "4": "Minimal hallucinations that don't impact the core answer",
        "3": "Some hallucinations that partially affect the answer quality",
        "2": "Significant hallucinations that undermine the answer",
        "1": "Severe hallucinations making the answer entirely unreliable"
    }
)

EVAL_METRICS_PROMPT_TEMPLATE = """\
You are an expert evaluator of question-answer pairs. Analyze the following Q&A pair and evaluate it objectively.

For this {{difficulty}} {{reasoning_type}} Q&A pair:
{{qa_pair}}

Take a deep breath and carefully evaluate each criterion based on the provided rubrics, considering the difficulty level and reasoning type indicated.
"""

# Add evaluation metrics column
aidd.add_column(
    name="eval_metrics",
    type="llm-judge",
    prompt=EVAL_METRICS_PROMPT_TEMPLATE,
    rubrics=[context_relevance_rubric, answer_precision_rubric, answer_completeness_rubric, hallucination_avoidance_rubric]
)

In [ ]:
# Preview a sample of generated records
preview = aidd.preview()

In [ ]:
preview.display_sample_record()

In [ ]:
# # Explore the generated preview as a Pandas DataFrame
# # ---------------------------
preview.dataset.df

In [ ]:
# Generate a report analyzing RAG coverage
# ---------------------

import json
from rich.console import Console
from rich.table import Table
from collections import Counter
import pandas as pd

def analyze_rag_coverage(df: pd.DataFrame) -> None:
    """
    Analyze the coverage of RAG evaluation examples with consistent formatting.

    Args:
        df: DataFrame containing RAG evaluation data
    """
    # Normalize the DataFrame
    qa_df = pd.json_normalize(
        df.assign(eval_metrics=lambda _df: _df["eval_metrics"].apply(
            lambda x: json.loads(x) if isinstance(x, str) else x
        )).to_dict(orient="records")
    )

    console = Console()
    categories = ['question_type', 'difficulty', 'reasoning_type']

    # Print header
    console.print("\n[bold blue]📊 RAG Evaluation Report[/bold blue]", justify="center")
    console.print("=" * 80, justify="center")
    console.print(f"\n[bold]Total Examples:[/bold] {len(qa_df)}")

    # Category distributions
    for category in categories:
        if category in qa_df.columns:
            # Count non-empty values
            counts = Counter(x for x in qa_df[category] if pd.notna(x) and x != '')
            if not counts:
                continue

            table = Table(title=f"\n{category.title()} Distribution")
            table.add_column("Category", style="cyan")
            table.add_column("Count", justify="right")
            table.add_column("Percentage", justify="right")

            total = sum(counts.values())
            for value, count in sorted(counts.items()):
                percentage = (count / total) * 100
                table.add_row(str(value), str(count), f"{percentage:.1f}%")

            console.print(table)

    # Extract evaluation metrics from the dataframe
    if 'eval_metrics' in df.columns:
        # Create a table for metrics
        metrics_table = Table(title="\nQuality Metrics Summary")
        metrics_table.add_column("Metric")
        metrics_table.add_column("Average Score", justify="right")
        
        # Get all metrics from the first row to understand structure
        metrics_dict = df['eval_metrics'][0]
        
        # Process each metric category
        for category, details in metrics_dict.items():
            if 'score' in details:
                # Convert score to float if it's stored as string
                try:
                    score_value = float(details['score'])
                    metrics_table.add_row(
                        category,
                        f"{score_value:.2f}/5.00"
                    )
                except (ValueError, TypeError):
                    # Handle case where score might not be a number
                    metrics_table.add_row(
                        category,
                        details['score']
                    )
        
        # Print the metrics table
        console.print(metrics_table)

# Usage
analyze_rag_coverage(preview.dataset.df)

In [14]:
# Generate and Analyze Full Dataset
# ---------------------------
# Uncomment these lines to generate evaluation pairs, save them to JSONL,
# and analyze the coverage and quality of the generated dataset.

workflow_run = aidd.create(
   num_records=100,
   name="rag_eval_generation",
   wait_for_completion=True
)

In [15]:
print("\nGenerated dataset shape:", workflow_run.dataset.df.shape)

analyze_rag_coverage(workflow_run.dataset.df)

workflow_run.dataset.df.to_json('rag_evals.jsonl', orient='records', lines=True)